In [1]:
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Read the CSV file into a DataFrame
df = pd.read_csv(
    "Constructions_Universe_AMBA_2023_-_Constructions_Universe_AMBA_2023.csv"
)

# Display the first 5 rows
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

# Print the column names and their data types
print(df.info())

| Construction                                                      | Program   | TYPE OF CONSTRUCTION   | STATUS    | Province     | Municipality     | Location            | Households   | LCS    | Sol Hab    | Executor     | AFIS   | AFIN   | Number of Main Exp.                | N° of private agreement   | Signature date ACU   | Latitude     | Years    | Length       | Long     | LATITUDE AND LONGITUDE   | UVI amount  nation total    | UVI total amount counterpart   | Remaining balance at UVIS   | Act of contruction start    |
|:------------------------------------------------------------------|:----------|:-----------------------|:----------|:-------------|:-----------------|:--------------------|:-------------|:-------|:-----------|:-------------|:-------|:-------|:-----------------------------------|:--------------------------|:---------------------|:-------------|:---------|:-------------|:---------|:-------------------------|:----------------------------|:-----------------------

In [2]:
# Filter out null values in `Remaining balance at UVIS` and values equal to '0'.
df_filtered = df[
    df["Remaining balance at UVIS"].notnull() & (df["Remaining balance at UVIS"] != "0")
]

# Filter out null values from the columns `UVI amount  nation total`, and `UVI total amount counterpart`.
df_filtered = df_filtered[
    df_filtered["UVI amount  nation total"].notnull()
    & df_filtered["UVI total amount counterpart"].notnull()
].copy()

# Remove commas from the columns `UVI amount  nation total`, and `UVI total amount counterpart` and convert to numeric datatype.
df_filtered["UVI amount  nation total"] = (
    df_filtered["UVI amount  nation total"]
    .astype(str)
    .str.replace(",", "", regex=False)
)
df_filtered["UVI total amount counterpart"] = (
    df_filtered["UVI total amount counterpart"]
    .astype(str)
    .str.replace(",", "", regex=False)
)

df_filtered["UVI amount  nation total"] = pd.to_numeric(
    df_filtered["UVI amount  nation total"]
)
df_filtered["UVI total amount counterpart"] = pd.to_numeric(
    df_filtered["UVI total amount counterpart"]
)

# Calculate deciles for `Remaining balance at UVIS`
remaining_balance_deciles = (
    df_filtered["Remaining balance at UVIS"]
    .quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
    .tolist()
)

# Add new column `remaining_balance_decile` based on the deciles
df_filtered["remaining_balance_decile"] = pd.cut(
    df_filtered["Remaining balance at UVIS"],
    bins=[-float("inf")] + remaining_balance_deciles + [float("inf")],
    labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
)

# Group by relevant columns, calculate mean of `Remaining balance at UVIS`, and sort
grouped_by_type = (
    df_filtered.groupby("TYPE OF CONSTRUCTION")["Remaining balance at UVIS"]
    .mean()
    .sort_values(ascending=False)
)
grouped_by_status = (
    df_filtered.groupby("STATUS")["Remaining balance at UVIS"]
    .mean()
    .sort_values(ascending=False)
)
grouped_by_province = (
    df_filtered.groupby("Province")["Remaining balance at UVIS"]
    .mean()
    .sort_values(ascending=False)
)
grouped_by_municipality = (
    df_filtered.groupby("Municipality")["Remaining balance at UVIS"]
    .mean()
    .sort_values(ascending=False)
)

# Compute the correlation
correlation = df_filtered[
    [
        "Remaining balance at UVIS",
        "UVI amount  nation total",
        "UVI total amount counterpart",
    ]
].corr()

# Print the results
print("Average remaining balance by construction type:\n")
print(grouped_by_type.to_markdown(numalign="left", stralign="left"))

print("\n\nAverage remaining balance by status:\n")
print(grouped_by_status.to_markdown(numalign="left", stralign="left"))

print("\n\nAverage remaining balance by province:\n")
print(grouped_by_province.to_markdown(numalign="left", stralign="left"))

print("\n\nAverage remaining balance by municipality:\n")
print(grouped_by_municipality.to_markdown(numalign="left", stralign="left"))

print("\n\nCorrelation between remaining balance and UVI amounts:\n")
print(correlation.to_markdown(numalign="left", stralign="left"))

KeyError: 'UVI amount  nation total'

In [3]:
# Print the column names
print(df.columns)

Index(['Construction', 'Program', 'TYPE OF CONSTRUCTION', 'STATUS', 'Province',
       'Municipality', 'Location', 'Households', 'LCS ', 'Sol Hab ',
       'Executor', 'AFIS', 'AFIN', 'Number of Main Exp. ',
       'N° of private agreement', 'Signature date ACU', 'Latitude ', 'Years ',
       'Length', 'Long ', 'LATITUDE AND LONGITUDE',
       'UVI amount  nation total ', 'UVI total amount counterpart',
       'Remaining balance at UVIS', 'Act of contruction start '],
      dtype='object')


In [5]:
import altair as alt

# Remove the trailing whitespaces from the relevant columns
df.columns = df.columns.str.rstrip()

# Filter out null values in `Remaining balance at UVIS` and values equal to '0'.
df_filtered = df[
    df["Remaining balance at UVIS"].notnull() & (df["Remaining balance at UVIS"] != "0")
]

# Filter out null values from the columns `UVI amount  nation total`, and `UVI total amount counterpart`.
df_filtered = df_filtered[
    df_filtered["UVI amount  nation total"].notnull()
    & df_filtered["UVI total amount counterpart"].notnull()
].copy()

# Remove commas from the columns `UVI amount  nation total`, and `UVI total amount counterpart` and convert to numeric datatype.
df_filtered["UVI amount  nation total"] = (
    df_filtered["UVI amount  nation total"]
    .astype(str)
    .str.replace(",", "", regex=False)
)
df_filtered["UVI total amount counterpart"] = (
    df_filtered["UVI total amount counterpart"]
    .astype(str)
    .str.replace(",", "", regex=False)
)

df_filtered["UVI amount  nation total"] = pd.to_numeric(
    df_filtered["UVI amount  nation total"]
)
df_filtered["UVI total amount counterpart"] = pd.to_numeric(
    df_filtered["UVI total amount counterpart"]
)

# Calculate deciles for `Remaining balance at UVIS`
remaining_balance_deciles = (
    df_filtered["Remaining balance at UVIS"]
    .quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
    .tolist()
)

# Add new column `remaining_balance_decile` based on the deciles
df_filtered["remaining_balance_decile"] = pd.cut(
    df_filtered["Remaining balance at UVIS"],
    bins=[-float("inf")] + remaining_balance_deciles + [float("inf")],
    labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
)

# Group by relevant columns, calculate mean of `Remaining balance at UVIS`, and sort
grouped_by_type = (
    df_filtered.groupby("TYPE OF CONSTRUCTION")["Remaining balance at UVIS"]
    .mean()
    .sort_values(ascending=False)
)
grouped_by_status = (
    df_filtered.groupby("STATUS")["Remaining balance at UVIS"]
    .mean()
    .sort_values(ascending=False)
)
grouped_by_province = (
    df_filtered.groupby("Province")["Remaining balance at UVIS"]
    .mean()
    .sort_values(ascending=False)
)
grouped_by_municipality = (
    df_filtered.groupby("Municipality")["Remaining balance at UVIS"]
    .mean()
    .sort_values(ascending=False)
)

# Compute the correlation
correlation = df_filtered[
    [
        "Remaining balance at UVIS",
        "UVI amount  nation total",
        "UVI total amount counterpart",
    ]
].corr()

# Print the results
print("Average remaining balance by construction type:\n")
print(grouped_by_type.to_markdown(numalign="left", stralign="left"))

print("\n\nAverage remaining balance by status:\n")
print(grouped_by_status.to_markdown(numalign="left", stralign="left"))

print("\n\nAverage remaining balance by province:\n")
print(grouped_by_province.to_markdown(numalign="left", stralign="left"))

print("\n\nAverage remaining balance by municipality:\n")
print(grouped_by_municipality.to_markdown(numalign="left", stralign="left"))

print("\n\nCorrelation between remaining balance and UVI amounts:\n")
print(correlation.to_markdown(numalign="left", stralign="left"))

# Plot the average remaining balance by construction type
chart1 = (
    alt.Chart(
        grouped_by_type.reset_index(),
        title="Average Remaining Balance by Construction Type",
    )
    .mark_bar()
    .encode(
        x=alt.X("TYPE OF CONSTRUCTION", sort=None),
        y="Remaining balance at UVIS",
        tooltip=["TYPE OF CONSTRUCTION", "Remaining balance at UVIS"],
    )
    .interactive()
)

chart1.save("remaining_balance_by_construction_type_bar_chart.json")

# Plot the average remaining balance by status
chart2 = (
    alt.Chart(
        grouped_by_status.reset_index(), title="Average Remaining Balance by Status"
    )
    .mark_bar()
    .encode(
        x=alt.X("STATUS", sort=None),
        y="Remaining balance at UVIS",
        tooltip=["STATUS", "Remaining balance at UVIS"],
    )
    .interactive()
)

chart2.save("remaining_balance_by_status_bar_chart.json")

ValueError: Unable to parse string "11.964.38681" at position 70

In [6]:
import altair as alt
import numpy as np

# Remove the trailing whitespaces from the relevant columns
df.columns = df.columns.str.rstrip()

# Remove '.' from the columns `UVI amount  nation total`
df_filtered["UVI amount  nation total"] = (
    df_filtered["UVI amount  nation total"]
    .astype(str)
    .str.replace(".", "", regex=False)
)

# Filter out null values in `Remaining balance at UVIS` and values equal to '0'.
df_filtered = df[
    df["Remaining balance at UVIS"].notnull() & (df["Remaining balance at UVIS"] != "0")
]

# Filter out null values from the columns `UVI amount  nation total`, and `UVI total amount counterpart`.
df_filtered = df_filtered[
    df_filtered["UVI amount  nation total"].notnull()
    & df_filtered["UVI total amount counterpart"].notnull()
].copy()

# Remove commas from the columns `UVI amount  nation total`, and `UVI total amount counterpart` and convert to numeric datatype.
df_filtered["UVI amount  nation total"] = (
    df_filtered["UVI amount  nation total"]
    .astype(str)
    .str.replace(",", "", regex=False)
)
df_filtered["UVI total amount counterpart"] = (
    df_filtered["UVI total amount counterpart"]
    .astype(str)
    .str.replace(",", "", regex=False)
)

df_filtered["UVI amount  nation total"] = pd.to_numeric(
    df_filtered["UVI amount  nation total"]
)
df_filtered["UVI total amount counterpart"] = pd.to_numeric(
    df_filtered["UVI total amount counterpart"]
)

# Calculate deciles for `Remaining balance at UVIS`
remaining_balance_deciles = (
    df_filtered["Remaining balance at UVIS"]
    .quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
    .tolist()
)

# Add new column `remaining_balance_decile` based on the deciles
df_filtered["remaining_balance_decile"] = pd.cut(
    df_filtered["Remaining balance at UVIS"],
    bins=[-float("inf")] + remaining_balance_deciles + [float("inf")],
    labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
)

# Group by relevant columns, calculate mean of `Remaining balance at UVIS`, and sort
grouped_by_type = (
    df_filtered.groupby("TYPE OF CONSTRUCTION")["Remaining balance at UVIS"]
    .mean()
    .sort_values(ascending=False)
)
grouped_by_status = (
    df_filtered.groupby("STATUS")["Remaining balance at UVIS"]
    .mean()
    .sort_values(ascending=False)
)
grouped_by_province = (
    df_filtered.groupby("Province")["Remaining balance at UVIS"]
    .mean()
    .sort_values(ascending=False)
)
grouped_by_municipality = (
    df_filtered.groupby("Municipality")["Remaining balance at UVIS"]
    .mean()
    .sort_values(ascending=False)
)

# Compute the correlation
correlation = df_filtered[
    [
        "Remaining balance at UVIS",
        "UVI amount  nation total",
        "UVI total amount counterpart",
    ]
].corr()

# Print the results
print("Average remaining balance by construction type:\n")
print(grouped_by_type.to_markdown(numalign="left", stralign="left"))

print("\n\nAverage remaining balance by status:\n")
print(grouped_by_status.to_markdown(numalign="left", stralign="left"))

print("\n\nAverage remaining balance by province:\n")
print(grouped_by_province.to_markdown(numalign="left", stralign="left"))

print("\n\nAverage remaining balance by municipality:\n")
print(grouped_by_municipality.to_markdown(numalign="left", stralign="left"))

print("\n\nCorrelation between remaining balance and UVI amounts:\n")
print(correlation.to_markdown(numalign="left", stralign="left"))

# Plot the average remaining balance by construction type
chart1 = (
    alt.Chart(
        grouped_by_type.reset_index(),
        title="Average Remaining Balance by Construction Type",
    )
    .mark_bar()
    .encode(
        x=alt.X("TYPE OF CONSTRUCTION", sort=None),
        y="Remaining balance at UVIS",
        tooltip=["TYPE OF CONSTRUCTION", "Remaining balance at UVIS"],
    )
    .interactive()
)

chart1.save("remaining_balance_by_construction_type_bar_chart.json")

# Plot the average remaining balance by status
chart2 = (
    alt.Chart(
        grouped_by_status.reset_index(), title="Average Remaining Balance by Status"
    )
    .mark_bar()
    .encode(
        x=alt.X("STATUS", sort=None),
        y="Remaining balance at UVIS",
        tooltip=["STATUS", "Remaining balance at UVIS"],
    )
    .interactive()
)

chart2.save("remaining_balance_by_status_bar_chart.json")

# Plot the histogram of the remaining balance
chart3 = (
    alt.Chart(df_filtered, title="Distribution of Remaining Balance")
    .mark_bar()
    .encode(
        x=alt.X(
            "Remaining balance at UVIS:Q", bin=True, title="Remaining Balance at UVIS"
        ),
        y=alt.Y("count()", title="Count"),
        tooltip=[
            alt.Tooltip(
                "Remaining balance at UVIS:Q",
                bin=True,
                title="Remaining Balance at UVIS",
            ),
            "count()",
        ],
    )
    .interactive()
)

chart3.save("remaining_balance_histogram.json")

# Plot scatterplots between `Remaining balance at UVIS` and `UVI amount  nation total`
chart4 = (
    alt.Chart(df_filtered, title="Remaining Balance vs. UVI amount nation total")
    .mark_circle()
    .encode(
        x="UVI amount  nation total",
        y="Remaining balance at UVIS",
        tooltip=["UVI amount  nation total", "Remaining balance at UVIS"],
    )
    .interactive()
)

chart4.save("remaining_balance_vs_nation_total_scatterplot.json")

# Plot scatterplots between `Remaining balance at UVIS` and `UVI total amount counterpart`
chart5 = (
    alt.Chart(df_filtered, title="Remaining Balance vs. UVI total amount counterpart")
    .mark_circle()
    .encode(
        x="UVI total amount counterpart",
        y="Remaining balance at UVIS",
        tooltip=["UVI total amount counterpart", "Remaining balance at UVIS"],
    )
    .interactive()
)

chart5.save("remaining_balance_vs_counterpart_scatterplot.json")

ValueError: Unable to parse string "11.964.38681" at position 70